In [13]:
# 네이버 테마 정리

import pandas as pd
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import requests
import time
from tqdm import tqdm
from datetime import datetime
import numpy as np
import xlwings as xw
import os

today = (datetime.today()).strftime('%Y%m%d')

df_theme = pd.DataFrame() # 테마리스트
df_theme_stock = pd.DataFrame() # 테마 안에 종목들 리스트

for i in tqdm(range(1,9)):
    url = f'https://finance.naver.com/sise/theme.naver?&page={i}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    data = soup.find('table',{'class':'type_1 theme'})
    make = parser_functions.make2d(data)
    table = pd.DataFrame(make,columns=make[0]).drop(index=0)
    table['테마명'].replace('',np.nan,inplace=True)
    table.dropna(inplace=True)
    table = table[['테마명','전일대비']]
    table.insert(0,'날짜',today)
    df_theme = pd.concat([df_theme,table],ignore_index=True)

    posts = soup.select('td.col_type1')

    for post in posts:
        link = 'https://finance.naver.com'
        theme_name = post.select_one('td.col_type1 >a').text.strip()
        theme_link = post.select_one('td.col_type1 >a').attrs['href']

        sub_url = link + theme_link
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text,'html.parser')
        sub_data = sub_soup.find('table',{'class':'type_5'})
        sub_make = parser_functions.make2d(sub_data)
        sub_table = pd.DataFrame(sub_make,columns=sub_make[0]).drop(index=0)
        sub_table = sub_table[['종목명', '현재가', '등락률','거래량', '거래대금','전일거래량']]
        sub_table.columns = ['종목명','내용','현재가', '등락률','거래량', '거래대금','전일거래량']
        sub_table = sub_table[['종목명', '현재가', '등락률','거래량', '거래대금','전일거래량']]
        sub_table['종목명'].replace('',np.nan,inplace=True)
        sub_table.dropna(inplace=True)
        sub_table.insert(0,'테마',theme_name)
        sub_table.insert(0,'날짜',today)
        df_theme_stock = pd.concat([df_theme_stock,sub_table],ignore_index=True)
        time.sleep(1)

if '테마.xlsx' in os.listdir():
    wb = xw.Book('테마.xlsx')
    ws = wb.sheets('테마리스트')
    ws_1 = wb.sheets('테마종목')
else:
    wb = xw.Book()
    ws = wb.sheets.add('테마리스트')
    ws_1 = wb.sheets.add('테마종목')

ws.activate()
ws.range('B2').value = df_theme
ws_1.activate()
ws_1.range('B2').value = df_theme_stock

wb.save()

100%|██████████| 8/8 [05:14<00:00, 39.30s/it]


In [22]:
import pandas as pd
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import requests
import time
from tqdm import tqdm
from datetime import datetime
import numpy as np
import xlwings as xw
import os


today = (datetime.today()).strftime('%Y-%m-%d')
url = f'https://finance.naver.com/sise/theme.naver?&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
data = soup.find('table',{'class':'type_1 theme'})
make = parser_functions.make2d(data)
table = pd.DataFrame(make,columns=make[0]).drop(index=0)
table['테마명'].replace('',np.nan,inplace=True)
table.dropna(inplace=True)
table = table[['테마명','전일대비']]
table['전일대비'] = table['전일대비'].str.replace('+','',regex=True).str.replace('%','',regex=True)
table = table.rename(columns={'전일대비':today})
table


,테마명,2024-01-27
3,2024 상반기 신규상장,44.53
4,조선기자재,11.69
5,밥솥,6.85
6,선박평형수 처리장치,5.45
7,초전도체,5.15
11,전기자전거,5.12
12,바이오인식(생체인식),3.72
13,건설 대표주,3.53
14,2차전지(생산),3.52
15,가상현실(VR),3.45
